# Etiquetado versión 2

En esta ocasión, exploraremos el índice de abandono de los alumnos considerando si entregaron el proyecto de programación de la asignatura. 

Para ello, seguiremos los siguientes pasos: 

- Obtener las tareas asociadas a la entrega del proyecto de programación, en este caso la 8 y la final del proyecto.
- Obtener todas las entregas que se hicieron sobre estos proyectos
- Hacer un left join desde la tabla de alumnos con estas entregas, aquellos que tengan valor null, serán aquellos que no tengan entregas asociadas
- Etiquetar los alumnos en base a este criterio y visualizar los resultados para verificar el balanceo de las clases. 

## Configuración de la sesión

In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import count
from pyspark.sql.functions import from_unixtime, date_format
import os

ruta_origen = "/home/carlos/Documentos/TFG/spark-workspace/data/raw/"
ruta_destino = "/home/carlos/Documentos/TFG/spark-workspace/data/etiquetas"
courseid_ip = 8683
os.makedirs(ruta_destino, exist_ok=True)

# Crear sesión Spark
spark = SparkSession.builder \
    .appName("Etiquetado versión 2") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

## Obtención de tareas de entrega de proyecto
 
Se parte de la siguiente información recogida sobre las tareas de la asignatura ip en el filtrado de datos de las tareas: 

```
Identificador, nombre y fecha de entrega de las tareas con al menos una entrega tras unir con los alumnos matriculados
+----------+-----------------------------------------------------------------------+-----------------+
|assignment|name                                                                   |duedate_formatted|
+----------+-----------------------------------------------------------------------+-----------------+
|7592      |Actividad 00. Actualización del perfil en el campus virtual            |25/09/23         |
|107688    |Entrega actividad 07                                                   |20/11/23         |
|108682    |Entrega de la actividad 08 - Proyecto - Parejas - NO VALE              |11/12/23         |
|108640    |Entrega de la actividad 08 - Realizada en pareja                       |11/12/23         |
|108641    |Entrega de la actividad 08 - Realizada individualmente                 |11/12/23         |
|113317    |Entrega del proyecto de programación Gomoku                            |27/05/24         |
|109226    |Entrega del proyecto de programación Gomoku - Realizado en pareja      |24/01/24         |
|113721    |Entrega del proyecto de programación Gomoku - Realizado en pareja      |01/07/24         |
|108047    |Entrega del proyecto de programación Gomoku - Realizado individualmente|24/01/24         |
|113720    |Entrega del proyecto de programación Gomoku - Realizado individualmente|01/07/24         |
+----------+-----------------------------------------------------------------------+-----------------+
```

Por lo tanto, obtendremos solo aquellas tareas asociadas a las entregas del proyecto que tuvieron lugar durante el primer cuatrimestre, siendo estas `108640, 108641, 109226, 108047 `

In [35]:
from pyspark.sql.functions import to_date, from_unixtime

df_tareas = spark.read.parquet(ruta_origen + "assign_cmi.parquet")

lista_id_tareas = [113721, 113720]
# lista_id_tareas = [108640, 108641]
df_tareas_proyecto = df_tareas.filter(col("id").isin(lista_id_tareas)) \
.select("id", "course", "name", 
        to_date(from_unixtime(col("allowsubmissionsfromdate"))).alias("allowsubmissionsfromdate"), 
        to_date(from_unixtime(col("duedate"))).alias("duedate"),
        to_date(from_unixtime(col("timemodified"))).alias("timemodified")) 

df_tareas_proyecto.show(truncate=False)



+------+------+-----------------------------------------------------------------------+------------------------+----------+------------+
|id    |course|name                                                                   |allowsubmissionsfromdate|duedate   |timemodified|
+------+------+-----------------------------------------------------------------------+------------------------+----------+------------+
|113720|8683  |Entrega del proyecto de programación Gomoku - Realizado individualmente|2024-06-13              |2024-07-01|2024-06-13  |
|113721|8683  |Entrega del proyecto de programación Gomoku - Realizado en pareja      |2024-06-13              |2024-07-01|2024-07-01  |
+------+------+-----------------------------------------------------------------------+------------------------+----------+------------+



## Obtener entregas hechas sobre estas tareas

In [36]:
df_entregas = spark.read.parquet(ruta_origen + "assign_submission_cmi.parquet")

# Obtenemos las entregas de las tareas del proyecto
df_entregas_proyecto = df_tareas_proyecto.join(df_entregas, df_tareas_proyecto.id == df_entregas.assignment, "inner").select("assignment", "name" , "userid", "status","duedate",  df_entregas.timemodified)



# Ahora que ya tenemos concatenadas estas tareas con las tareas que se han hecho de ellas, podemos ver un repaso de cuantas entregas hubo para cada una 
entregas_por_tarea = df_entregas_proyecto.groupBy("assignment", "name", "duedate" ).agg(count("*").alias("num_entregas")).orderBy("duedate")

entregas_por_tarea.show(truncate=False)


+----------+-----------------------------------------------------------------------+----------+------------+
|assignment|name                                                                   |duedate   |num_entregas|
+----------+-----------------------------------------------------------------------+----------+------------+
|113720    |Entrega del proyecto de programación Gomoku - Realizado individualmente|2024-07-01|91          |
|113721    |Entrega del proyecto de programación Gomoku - Realizado en pareja      |2024-07-01|110         |
+----------+-----------------------------------------------------------------------+----------+------------+



Parece un poco extraño que la suma de las entregas individuales y por parejas sume un número mayor que el número de alumnos matriculados en el curso


In [37]:
from pyspark.sql.functions import sum

df_alumnos_ip = spark.read.parquet(ruta_origen + "alumnos_ip_cmi.parquet") 

print(f"Número de alumnos: {df_alumnos_ip.count()}")

entregas_por_tarea.groupBy("duedate").agg(sum(col("num_entregas")).alias("suma_ind_parejas")).show(truncate=False)

Número de alumnos: 201
+----------+----------------+
|duedate   |suma_ind_parejas|
+----------+----------------+
|2024-07-01|201             |
+----------+----------------+



## Obtener qué alumnos  hicieron estas entregas y cuáles no 

In [38]:
from pyspark.sql.functions import when, col, lit

df_usuarios_con_entrega = df_entregas_proyecto.select("userid").distinct()

df_alumnos_con_entrega = df_alumnos_ip.join(
    df_usuarios_con_entrega.withColumn("entrego_proyecto", lit(True)),
    on="userid",
    how="left"
)
df_alumnos_con_entrega = df_alumnos_con_entrega.withColumn(
    "entrego_proyecto",
    when(col("entrego_proyecto").isNull(), False).otherwise(True)
)

num_no_entregaron = df_alumnos_con_entrega.filter(col("entrego_proyecto") == False).count()
print(f"Número de alumnos: {df_alumnos_ip.count()}")
print(f"Número de alumnos que no entregaron el proyecto: {num_no_entregaron}")


df_alumnos_con_entrega.show(truncate=False)


Número de alumnos: 201
Número de alumnos que no entregaron el proyecto: 61
+----------------------------------------------------------------+----------------+
|userid                                                          |entrego_proyecto|
+----------------------------------------------------------------+----------------+
|3b8d431cbee3182d06225f9d5ab51f5806e8042f145e0ad0e37d98f56ae78f3d|false           |
|368093a57fe640879a9fc57ecb7e2c846b7dadf19620bfc9c4c001daeaf9af0f|true            |
|b10bfb431460e96f99322635d8c44d0fb4f74302c61334119ab1c842e049fac1|true            |
|e86f7c2d50f4fa9ee5b3f8bf9a25af4c9ef573fa28a0c8c50df815971efa0529|true            |
|bc941ebd72011adc6a5fbaa15a85c36e9dd20fd0af0156317b764f98a7a099ee|true            |
|d46525fcffba400caa4fc21005b92fa32ed6922cf520d4879aa155394ee81d54|true            |
|f6be71259bf65d67b39b33c9d0cc4859a713d8ab8a3941b1ea0fbb7bc2f1acaa|true            |
|8cd597a8e1181f4db4ce03e1d02e59e4b5798921adeaee796526d96cc1975417|false           |
|